# Import and install required dependencies

In [ ]:
pip install ecdsa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 7.0 MB/s eta 0:00:00


In [ ]:
import hashlib
import time
import base64
from abc import ABC, abstractmethod
import ecdsa
from ecdsa.keys import VerifyingKey, SigningKey
from typing import Tuple, List
import sys, json
import os

# 1. Implement a minimal Blockchain in Python

## 1.1 CryptoProvider Interface and ECDSA / SHA256 Implementation


**Create the `CryptoProvider` Interface**

In [ ]:
class CryptoProvider(ABC):
    """
    Abstract base class for cryptographic operations.

    This class provides a standardized interface for cryptographic operations,
    including key generation, message signing, signature verification, and data hashing.
    Subclasses must implement each method to perform the specified cryptographic functionality.
    """

    def __repr__(self) -> str:
        """
        Return a string representation of the provider class.

        Returns:
            str: A string representation indicating the class name.
        """
        return "ECDSASHA256Provider"

    @abstractmethod
    def generate_key_pair(self) -> Tuple[bytes, bytes]:
        """
        Generate a cryptographic key pair.

        Returns:
            Tuple[bytes, bytes]: A tuple containing the private and public keys as bytes.
        """
        pass

    @abstractmethod
    def sign(self, private_key: bytes, message: str) -> bytes:
        """
        Sign a message using the provided private key.

        Args:
            private_key (bytes): The private key used to sign the message.
            message (str): The message to be signed.

        Returns:
            bytes: The generated signature as bytes.
        """
        pass

    @abstractmethod
    def verify(self, public_key: bytes, message: str, signature: bytes) -> bool:
        """
        Verify a signature for a given message and public key.

        Args:
            public_key (bytes): The public key used to verify the signature.
            message (str): The message whose signature is to be verified.
            signature (bytes): The signature to verify.

        Returns:
            bool: True if the signature is valid; False otherwise.
        """
        pass

    @abstractmethod
    def hash(self, data: str) -> str:
        """
        Compute a cryptographic hash of the provided data.

        Args:
            data (str): The data to be hashed.

        Returns:
            str: The resulting hash as a hexadecimal string.
        """
        pass

**Create the ECDSA / SHA256 Implementation `ECDSASHA256Provider`**

In [ ]:
class ECDSASHA256Provider(CryptoProvider):
    """
    Provides cryptographic operations using ECDSA with SHA-256 hashing.

    This class implements the `CryptoProvider` interface, using the SECP256k1 elliptic curve
    for ECDSA key generation, message signing, and signature verification, along with SHA-256
    hashing for data integrity.
    """

    def generate_key_pair(self) -> Tuple[bytes, bytes]:
        """
        Generate a cryptographic ECDSA key pair.

        Uses the SECP256k1 elliptic curve to generate a private and public key pair.

        Returns:
            Tuple[bytes, bytes]: A tuple containing the private and public keys as DER-encoded bytes.
        """
        private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
        public_key = private_key.get_verifying_key()
        return private_key.to_der(), public_key.to_der()

    def sign(self, private_key: bytes, message: str) -> bytes:
        """
        Sign a message using the provided ECDSA private key.

        Args:
            private_key (bytes): The private key in DER format.
            message (str): The message to be signed.

        Returns:
            bytes: The generated signature as bytes.
        """
        return ecdsa.SigningKey.from_der(private_key).sign(message.encode('utf-8'))

    def verify(self, public_key: bytes, message: str, signature: bytes) -> bool:
        """
        Verify a signature for a given message and ECDSA public key.

        Args:
            public_key (bytes): The public key in DER format used to verify the signature.
            message (str): The message whose signature is to be verified.
            signature (bytes): The signature to verify.

        Returns:
            bool: True if the signature is valid; False otherwise.
        """
        try:
            return ecdsa.VerifyingKey.from_der(public_key).verify(signature, message.encode('utf-8'))
        except ecdsa.BadSignatureError:
            return False

    def hash(self, data: str) -> str:
        """
        Compute a SHA-256 cryptographic hash of the provided data.

        Args:
            data (str): The data to be hashed.

        Returns:
            str: The resulting SHA-256 hash as a hexadecimal string.
        """
        return hashlib.sha256(data.encode('utf-8')).hexdigest()


## 1.2 Create the Blockchain

**Create the `Transaction` class**

In [ ]:
class Transaction():
    """
    Represents a blockchain transaction between a sender and a recipient.

    This class contains the details of a transaction, including the sender, recipient,
    amount, and a cryptographic provider for signing and verifying the transaction.
    """

    def __init__(self, sender: str, recipient: str, amount: int, crypto_provider: CryptoProvider):
        """
        Initialize a new transaction.

        Args:
            sender (str): The sender's public key as a base64-encoded string.
            recipient (str): The recipient's public key as a base64-encoded string.
            amount (int): The amount of funds to transfer.
            crypto_provider (CryptoProvider): The cryptographic provider used for signing and verifying the transaction.
        """
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.crypto_provider = crypto_provider
        self.signature = None

    def sign_transaction(self, private_key: bytes):
        """
        Sign the transaction using the sender's private key.

        Args:
            private_key (bytes): The sender's private key in bytes format.
        """
        transaction_string = f'{self.sender}{self.recipient}{self.amount}{self.crypto_provider}'
        self.signature = self.crypto_provider.sign(private_key, transaction_string)

    def is_valid(self) -> bool:
        """
        Verify the validity of the transaction signature.

        Returns:
            bool: True if the transaction signature is valid; False otherwise.
        """
        transaction_string = f'{self.sender}{self.recipient}{self.amount}{self.crypto_provider}'
        return self.crypto_provider.verify(base64.b64decode(self.sender), transaction_string, self.signature)

**Create the `Block` class**

In [ ]:
class Block():
    """
    Represents a block in a blockchain.

    Each block contains a list of transactions, a reference to the previous block's hash,
    and is associated with a cryptographic provider for hashing and verification. The block
    includes metadata such as a timestamp, a nonce for proof-of-work, and its own computed hash.
    """

    def __init__(self, index: int, previous_hash: str, transactions: List[Transaction], crypto_provider: CryptoProvider, timestamp=None):
        """
        Initialize a new block.

        Args:
            index (int): The index of the block in the blockchain.
            previous_hash (str): The hash of the previous block in the chain.
            transactions (List[Transaction]): A list of transactions included in the block.
            crypto_provider (CryptoProvider): The cryptographic provider used for hashing and verification.
            timestamp (float, optional): The timestamp of block creation. Defaults to the current time.
        """
        self.index = index
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.crypto_provider = crypto_provider
        self.timestamp = timestamp or time.time()
        self.nonce = 0
        self.hash = None

    def compute_hash(self) -> str:
        """
        Compute the cryptographic hash of the block's contents (block's index, previous hash, transactions, cryptographic provider, timestamp,
        and nonce)

        Returns:
            str: The computed hash of the block as a hexadecimal string.
        """
        block_string = f'{self.index}{self.previous_hash}{self.transactions}{self.crypto_provider}{self.timestamp}{self.nonce}'
        return self.crypto_provider.hash(block_string)

    def is_valid(self) -> bool:
        """
        Verify the validity of the block. Checks that all transactions in the block are valid and that the block's hash matches
        the computed hash based on its current contents.

        Returns:
            bool: True if the block is valid; False otherwise.
        """
        if (
            not all(transaction.is_valid() for transaction in self.transactions) or
            self.hash != self.compute_hash()
        ):
            return False
        return True

**Create the `Blockchain` class**

In [ ]:
class Blockchain():
    """
    Represents a blockchain, which is a sequence of blocks containing transactions.

    The blockchain maintains a chain of blocks, manages pending transactions, and enables
    the mining process. It includes methods for validating the chain and retrieving balances
    for specific addresses.
    """

    def __init__(self, block_size: int, difficulty: int, crypto_provider: CryptoProvider):
        """
        Initialize a new blockchain.

        Args:
            block_size (int): The maximum number of transactions per block.
            difficulty (int): The difficulty level for the proof-of-work algorithm, determining the number of leading zeros required in the hash.
            crypto_provider (CryptoProvider): The cryptographic provider used for hashing and verification.
        """
        self.block_size = block_size
        self.difficulty = difficulty
        self.crypto_provider = crypto_provider
        self.chain = []
        self.pending_transactions = []
        self._create_genesis_block()

    def _create_genesis_block(self):
        """
        Create the genesis (first) block in the blockchain. The genesis block has no previous hash, no transactions, and
        is precomputed with a valid hash. Shouldnt' be called manually.
        """
        genesis_block = Block(0, "0", [], self.crypto_provider)
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    def add_transaction(self, transaction: Transaction):
        """
        Add a new transaction to the list of pending transactions.

        Args:
            transaction (Transaction): The transaction to be added to the blockchain.
        """
        self.pending_transactions.append(transaction)

    def mine_pending_transactions(self):
        """
        Mine the pending transactions and add a new block to the blockchain. Mines transactions up to the block size limit,
        computes the proof-of-work to meet the difficulty level, and adds the new block to the chain. Resets pending transactions
        after the block is mined.

        Returns:
            str: Message indicating if there were no transactions to mine.
        """
        if not self.pending_transactions:
            return "No transactions to mine."
        block = Block(len(self.chain), self.chain[-1].compute_hash(), self.pending_transactions[:self.block_size], self.crypto_provider)
        while not block.compute_hash().startswith('0' * self.difficulty):
            block.nonce += 1
        block.hash = block.compute_hash()
        self.chain.append(block)
        self.pending_transactions = self.pending_transactions[self.block_size:]

    def is_valid(self) -> bool:
        """
        Verify the integrity of the blockchain. Checks each block in the chain to ensure all blocks are valid and that the hashes
        link correctly to maintain the chain's integrity.

        Returns:
            bool: True if the blockchain is valid; False otherwise.
        """
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if (
                not current_block.is_valid() or
                current_block.previous_hash != previous_block.hash
            ):
                return False
        return True

    def get_balance(self, address):
        """
        Calculate the balance for a given address. Iterates through the blockchain to sum up all transactions related to the specified
        address to determine the current balance.

        Args:
            address (str): The address to calculate the balance for.

        Returns:
            int: The balance of the specified address.
        """
        balance = 0
        for block in self.chain:
            for transaction in block.transactions:
                if transaction.recipient == address:
                    balance += transaction.amount
                if transaction.sender == address:
                    balance -= transaction.amount
        return balance

**Add methods to display the full blockchain as a printable string**

In [ ]:
def convert_transaction_to_string(self) -> str:
    return (
        "\n"
        f"  | Sender:     {self.sender}\n"
        f"  | Recipient:  {self.recipient}\n"
        f"  | Amount:     {self.amount}\n"
        f"  | Signature:  {base64.b64encode(self.signature).decode('utf-8')}"
    )

Transaction.__str__ = convert_transaction_to_string
del convert_transaction_to_string

def convert_block_to_string(self) -> str:
        return (
            f"Block #{self.index}\n"
            f"Previous Hash: {self.previous_hash}\n"
            f"Transactions:" + "\n  | ".join(str(transaction) for transaction in self.transactions) + "\n"
            f"Timestamp: {self.timestamp}\n"
            f"Nonce: {self.nonce}\n"
            f"Hash: {self.hash}"
        )

Block.__str__ = convert_block_to_string
del convert_block_to_string

def convert_blockchain_to_string(self) -> str:
    return (
        f"Blockchain with Blocksize {self.block_size} and Difficulty {self.difficulty}\n\n"
        + "\n\n".join(str(block) for block in self.chain) + "\n"
        + "\nPending Transactions:" + "\n  | ".join(str(transaction) for transaction in self.pending_transactions) + "\n"
    )

Blockchain.__str__ = convert_blockchain_to_string
del convert_blockchain_to_string

## 1.3 Test the Blockchain

In [ ]:
crypto_provider = ECDSASHA256Provider()
private_key1, public_key1 = crypto_provider.generate_key_pair()
address1 = base64.b64encode(public_key1).decode("utf-8")
private_key2, public_key2 = crypto_provider.generate_key_pair()
address2 = base64.b64encode(public_key2).decode("utf-8")

transaction1 = Transaction(address1, address2, 30, crypto_provider)
transaction1.sign_transaction(private_key1)

transaction2 = Transaction(address2, address1, 10, crypto_provider)
transaction2.sign_transaction(private_key2)

transaction3 = Transaction(address2, address1, 15, crypto_provider)
transaction3.sign_transaction(private_key2)

blockchain = Blockchain(2, 3, crypto_provider)
blockchain.add_transaction(transaction1)
blockchain.add_transaction(transaction2)
blockchain.add_transaction(transaction3)
blockchain.mine_pending_transactions()

print(blockchain)

print("\nChecking the balances of the addresses...")
print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

blockchain.mine_pending_transactions()
print("\nMining the last pending transaction...")
print("Checking the balances of the addresses again...")
print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

print("\nChecking the validity of the blockchain...")
print(f"Blockchain validity: {blockchain.is_valid()}")

blockchain.chain[1].transactions[1].amount = 20
print("Modifying blockchain...")
print(f"Blockchain validity: {blockchain.is_valid()}")

Blockchain with Blocksize 2 and Difficulty 3

Block #0
Previous Hash: 0
Transactions:
Timestamp: 1731963543.1517699
Nonce: 0
Hash: 655cd6e888d5813680490d8786022163d829ce12d13afeda2476e46f218d019e

Block #1
Previous Hash: 655cd6e888d5813680490d8786022163d829ce12d13afeda2476e46f218d019e
Transactions:
  | Sender:     MFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEcDCCs0k2O6iVpn/m36lSrRU7XoLtYv1LT/m17/oalVO259NaPsXDa6nQG+1hWY1kNslqh8EOuu4AKmsC6eqZEg==
  | Recipient:  MFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEmFPypusHfcWJYkR7UN/j7GaTowoepPVPMUYf2Qc3vZ37+hPNmdlTEeIZOpzeqrwm4XQTXTCMfzQA5jOQLtqrgQ==
  | Amount:     30
  | Signature:  24g6fr5Q5/N8152FTCT6oVLOa1MDlTQXfRIC3nMdrHE+f5QJMqPJN+EHyPf7Vw65WG/eDWgFypCjPyYt0VMd8g==
  | 
  | Sender:     MFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEmFPypusHfcWJYkR7UN/j7GaTowoepPVPMUYf2Qc3vZ37+hPNmdlTEeIZOpzeqrwm4XQTXTCMfzQA5jOQLtqrgQ==
  | Recipient:  MFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEcDCCs0k2O6iVpn/m36lSrRU7XoLtYv1LT/m17/oalVO259NaPsXDa6nQG+1hWY1kNslqh8EOuu4AKmsC6eqZEg==
  | Amount:     10
  | 

In [ ]:
private_key1_size = sys.getsizeof(private_key1)
public_key1_size = sys.getsizeof(public_key1)
print(f"Private Key 1 Size: {private_key1_size} bytes")
print(f"Public Key 1 Size: {public_key1_size} bytes")

transaction1_signature_size = sys.getsizeof(transaction1.signature)
print(f"Transaction 1 Signature Size: {transaction1_signature_size} bytes")

blockchain_data = json.dumps([block.__dict__ for block in blockchain.chain], default=str).encode()
blockchain_size = sys.getsizeof(blockchain_data)
print(f"Blockchain Storage Size: {blockchain_size / 1024:.2f} KB")

Private Key 1 Size: 151 bytes
Public Key 1 Size: 121 bytes
Transaction 1 Signature Size: 97 bytes
Blockchain Storage Size: 1.99 KB


#3. Implement a selection of Post-Quantum-CryptoProviders for our Blockchain

## SPHINCS+

For SPHINCS+, there exists a Python package called pyspx.

Documentation is available here:
- https://github.com/sphincs/pyspx
- https://pypi.org/project/PySPX/

pyspx provides capabilites for generating public-private key pairs, signing and verifying messages.

Example usage: https://github.com/nakov/Practical-Cryptography-for-Developers-Book/blob/3cbe60554b9a1fce016a41fce708525adcd0323f/quantum-safe-cryptography/quantum-safe-signatures-example.md


SPHINCS+ uses tweakable hash functions. The official document (https://sphincs.org/data/sphincs+-specification.pdf) defines three different signature schemes which are obtained by instantiating the cryptographic function families of SPHINCS+ with SHA-256, SHAKE256, and Haraka.

The following implementation uses SHAKE256, as there is existing support in pyspx.

In [ ]:
pip install pyspx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.9 MB/s eta 0:00:00


In [ ]:
import pyspx.shake_128f as sphincs  # Assuming SPHINCS+ shake256 with 128-bit security

class SphincsProvider(CryptoProvider):
    """
    SphincsProvider implements the SPHINCS+ cryptographic operations for key generation,
    signing, signature verification, and hashing using the post-quantum secure SPHINCS+ method.
    """

    def __repr__(self) -> str:
        return "SphincsProvider"

    def generate_key_pair(self) -> Tuple[bytes, bytes]:
        """
        Generate a SPHINCS+ key pair.

        Returns:
            Tuple[bytes, bytes]: A tuple containing the private and public keys as bytes.
        """
        seed = os.urandom(sphincs.crypto_sign_SEEDBYTES)
        public_key, private_key = sphincs.generate_keypair(seed)
        return private_key, public_key

    def sign(self, private_key: bytes, message: str) -> bytes:
        """
        Sign a message using the provided SPHINCS+ private key.

        Args:
            private_key (bytes): The private key used to sign the message.
            message (str): The message to be signed.

        Returns:
            bytes: The generated signature as bytes.
        """
        message_bytes = message.encode('utf-8')
        signature = sphincs.sign(message_bytes, private_key)
        return signature

    def verify(self, public_key: bytes, message: str, signature: bytes) -> bool:
        """
        Verify a signature for a given message and SPHINCS+ public key.

        Args:
            public_key (bytes): The public key used to verify the signature.
            message (str): The message whose signature is to be verified.
            signature (bytes): The signature to verify.

        Returns:
            bool: True if the signature is valid; False otherwise.
        """
        message_bytes = message.encode('utf-8')
        try:
            return sphincs.verify(signature, message_bytes, public_key)
        except Exception:
            return False

    def hash(self, data: str) -> str:
        """
        Compute a cryptographic hash of the provided data using SHAKE-256.

        Args:
            data (str): The data to be hashed.

        Returns:
            str: The resulting hash as a hexadecimal string.
        """
        hasher = hashlib.shake_256()
        hasher.update(data.encode('utf-8'))
        return hasher.hexdigest(64)  # 64 hex digits for 256-bit output


In [ ]:
crypto_provider = SphincsProvider()
private_key1, public_key1 = crypto_provider.generate_key_pair()
address1 = base64.b64encode(public_key1).decode("utf-8")
private_key2, public_key2 = crypto_provider.generate_key_pair()
address2 = base64.b64encode(public_key2).decode("utf-8")

transaction1 = Transaction(address1, address2, 30, crypto_provider)
transaction1.sign_transaction(private_key1)

transaction2 = Transaction(address2, address1, 10, crypto_provider)
transaction2.sign_transaction(private_key2)

transaction3 = Transaction(address2, address1, 15, crypto_provider)
transaction3.sign_transaction(private_key2)

blockchain = Blockchain(2, 3, crypto_provider)
blockchain.add_transaction(transaction1)
blockchain.add_transaction(transaction2)
blockchain.add_transaction(transaction3)
blockchain.mine_pending_transactions()

print(blockchain)

print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

blockchain.mine_pending_transactions()

print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

print(f"Blockchain validity: {blockchain.is_valid()}")
blockchain.chain[1].transactions[1].amount = 20
print(f"Blockchain validity after modification: {blockchain.is_valid()}")


private_key1_size = sys.getsizeof(private_key1)
public_key1_size = sys.getsizeof(public_key1)
print(f"Private Key 1 Size: {private_key1_size} bytes")
print(f"Public Key 1 Size: {public_key1_size} bytes")

transaction1_signature_size = sys.getsizeof(transaction1.signature)
print(f"Transaction 1 Signature Size: {transaction1_signature_size} bytes")

blockchain_data = json.dumps([block.__dict__ for block in blockchain.chain], default=str).encode()
blockchain_size = sys.getsizeof(blockchain_data)
print(f"Blockchain Storage Size: {blockchain_size / 1024:.2f} KB")

Blockchain with Blocksize 2 and Difficulty 3

Block #0
Previous Hash: 0
Transactions:
Timestamp: 1732796012.7620676
Nonce: 0
Hash: 4cf2f14a308195261b54e9f5dabe6413767584e748f4b69185216a7f0a01340140a041ce0ba8d3baae5dad9a9546d5b916fa9bad1b63c22d010fb703a5f09621

Block #1
Previous Hash: 4cf2f14a308195261b54e9f5dabe6413767584e748f4b69185216a7f0a01340140a041ce0ba8d3baae5dad9a9546d5b916fa9bad1b63c22d010fb703a5f09621
Transactions:
  | Sender:     DwnpaQJTkya+k4+nfd7Ax5iJeAJsFNK3PAA9rD7ZFkA=
  | Recipient:  Gvo7ssAbQjT9RZTUrvPqZVYOseuhar2OiyaeRa3B228=
  | Amount:     30
  | Signature:  ...
  | 
  | Sender:     Gvo7ssAbQjT9RZTUrvPqZVYOseuhar2OiyaeRa3B228=
  | Recipient:  DwnpaQJTkya+k4+nfd7Ax5iJeAJsFNK3PAA9rD7ZFkA=
  | Amount:     10
  | Signature:  ...
Timestamp: 1732796012.7622051
Nonce: 1603
Hash: 0005e37a219d2984cf715c21f8329469185e3dbc54146062ec3fbad9e65d554b11ab75ea2ccc583445107a1a8944579bc32b6e342c549f4ed366ab6850de5776

Pending Transactions:
  | Sender:     Gvo7ssAbQjT9RZTUrvPqZVYOseuha

## Dilithium

https://pypi.org/project/dilithium/#description

In [ ]:
pip install dilithium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.6 MB/s eta 0:00:00


In [ ]:
from dilithium.dilithium import Dilithium2 as Dilithium

class DilithiumProvider(CryptoProvider):
    """
    DilithiumProvider implements the CRYSTALS-Dilithium cryptographic operations for key generation,
    signing, signature verification, and hashing using the post-quantum secure Dilithium method.
    """

    def __repr__(self) -> str:
        return "DilithiumProvider"

    def generate_key_pair(self) -> Tuple[bytes, bytes]:
        """
        Generate a Dilithium key pair.

        Returns:
            Tuple[bytes, bytes]: A tuple containing the private and public keys as bytes.
        """
        key_seed = os.urandom(32)
        public_key, private_key  = Dilithium.keygen(key_seed)
        return private_key, public_key

    def sign(self, private_key: bytes, message: str) -> bytes:
        """
        Sign a message using the provided Dilithium private key.

        Args:
            private_key (bytes): The private key used to sign the message.
            message (str): The message to be signed.

        Returns:
            bytes: The generated signature as bytes.
        """
        message_bytes = message.encode('utf-8')
        signature = Dilithium.sign_with_input(private_key, message_bytes)
        return signature

    def verify(self, public_key: bytes, message: str, signature: bytes) -> bool:
        """
        Verify a signature for a given message and Dilithium public key.

        Args:
            public_key (bytes): The public key used to verify the signature.
            message (str): The message whose signature is to be verified.
            signature (bytes): The signature to verify.

        Returns:
            bool: True if the signature is valid; False otherwise.
        """
        message_bytes = message.encode('utf-8')
        try:
            Dilithium.verify(public_key, message_bytes, signature)
            return True
        except Exception:
            return False

    def hash(self, data: str) -> str:
        """
        Compute a cryptographic hash of the provided data using SHA-256.

        Args:
            data (str): The data to be hashed.

        Returns:
            str: The resulting hash as a hexadecimal string.
        """
        hasher = hashlib.sha256()
        hasher.update(data.encode('utf-8'))
        return hasher.hexdigest()

In [ ]:
crypto_provider = DilithiumProvider()
private_key1, public_key1 = crypto_provider.generate_key_pair()
address1 = base64.b64encode(public_key1).decode("utf-8")
private_key2, public_key2 = crypto_provider.generate_key_pair()
address2 = base64.b64encode(public_key2).decode("utf-8")

transaction1 = Transaction(address1, address2, 30, crypto_provider)
transaction1.sign_transaction(private_key1)

transaction2 = Transaction(address2, address1, 10, crypto_provider)
transaction2.sign_transaction(private_key2)

transaction3 = Transaction(address2, address1, 15, crypto_provider)
transaction3.sign_transaction(private_key2)

blockchain = Blockchain(2, 3, crypto_provider)
blockchain.add_transaction(transaction1)
blockchain.add_transaction(transaction2)
blockchain.add_transaction(transaction3)
blockchain.mine_pending_transactions()

print(blockchain)

print("\nChecking the balances of the addresses...")
print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

blockchain.mine_pending_transactions()
print("\nMining the last pending transaction...")
print("Checking the balances of the addresses again...")
print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

print("\nChecking the validity of the blockchain...")
print(f"Blockchain validity: {blockchain.is_valid()}")

blockchain.chain[1].transactions[1].amount = 20
print("Modifying blockchain...")
print(f"Blockchain validity: {blockchain.is_valid()}")

tr =  b'\xa1\xa3\xbc\xb5\x9d\xcc\xdd\xdf~6c\xfb\x80\xcc7\x8eY\x0e\xf3\xfa\xbe\xea\x06\xc0#\x9d\x8ev\xe6\x91\xa0m'
m =  b'TuHqgtqxkyrBXyuR9Z82cowweDwHQOwe7zJTICAi+agzhLzxos3Z/aYc07mrhjPL6D1KlwNCd02Wa+8O3HcYGaRTCbw2Ae6MVPNYlQQRZIbf2ogK36Nti/3WLZTilBerAjH13+cni+zNiAsoG1jUEKdfYuTtTKhDLLkoy3J9Fd3xujazPbFHEw6Gf6Jc+Bep+XUcBP9tuDBzasPwNPeXDmODUYFqcBKXQi2wvM4vtmsz4jmwoB9qtmiM/oEaao5Pc4YGz93oJtADp5Y11+Rc2cjFqTMgUh5MQlO8XURGdcUnHUdvFiAgCA7hPg5TYmTykp28kn4eJLxzPejlrAKT9WXjUZlUfTzWZ3BNPoXGFUgAGVePL7eB+h5CJy+KCsQJsYNNYDq9iz4Wzzb/OwJY3psXOd7soIZsKBtk+mSNxg9OO6XQIDHNo/sZslwtzK+XZFkcrBjfYrC3j9OSdSKqwxnNlcWWUyP7Z/HXsQGYfZVdoGWkqRkYqf70CrnLuR7vGRinYBLeJPdpk3JmqRHo7NKlqr/nnGYq1Ti29eeEZ0TcBBp54CvHt84kP8rrgCOAxPo3Qs1jNqXQsxvddirEe1AzPTiwygerCzXpDezBbquVwlk7Y54T7KJm8ebqhqfrVr7Y5kq75Nqryni6dyk3F8SpZBHvPbcknjRuJBJhT/0cBRDjFiaujZEue4Vf8DO7yvRFZjISzK/Pfn0TRTzYCeBlBWh6eT3T/FZLxSSo7byTqCuHA7HcCbfc9SMcjtYOummCme5fLOcGBsUzLiZ537ISJwnZJFf3QAAm4IxdD/LtW2PmFbB0s9yB3CMNJOF2CWuBWcMCwjs8s2IgYRGG899vq5NSXurQ90w/oYMHoMX9R00

In [ ]:
private_key1_size = sys.getsizeof(private_key1)
public_key1_size = sys.getsizeof(public_key1)
print(f"Private Key 1 Size: {private_key1_size} bytes")
print(f"Public Key 1 Size: {public_key1_size} bytes")

transaction1_signature_size = sys.getsizeof(transaction1.signature)
print(f"Transaction 1 Signature Size: {transaction1_signature_size} bytes")

blockchain_data = json.dumps([block.__dict__ for block in blockchain.chain], default=str).encode()
blockchain_size = sys.getsizeof(blockchain_data)
print(f"Blockchain Storage Size: {blockchain_size / 1024:.2f} KB")

Private Key 1 Size: 2561 bytes
Public Key 1 Size: 1345 bytes
Transaction 1 Signature Size: 2453 bytes
Blockchain Storage Size: 20.74 KB


In [ ]:
pip install pqcrypto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pqcrypto: filename=pqcrypto-0.1.3-cp310-cp310-manylinux_2_35_x86_64.whl size=2119082 sha256=471e284861276cdb8d65ba2f7a65b0c506a26ddaa8ff575a5aab34652bd42f26
  Stored in directory: /root/.cache/pip/wheels/89/13/4f/b2c678a9f6c473acb0d259d47c30387bab4f836f28dab19f1c
Successfully built pqcrypto


In [ ]:
from pqcrypto.sign.dilithium4 import generate_keypair, sign, verify

ModuleNotFoundError: No module named 'pqcrypto._sign.dilithium4'